In [1]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian
from structurerl import * 

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
from modules.CMI import DR_CMI, CDL_CMI

In [5]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = create_dataset_DGP_binary_A_conf( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [6]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [7]:
def train_dr(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=N)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()
        #print(a)

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values

In [8]:
def train_cdl(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=N)
        #dataset = create_dataset_DGP_binary_A(Dim=Dim, N=64)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return cdl_est_values

In [9]:
N=64
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.66903359e-01 3.78592699e-01 1.09443370e+00 3.73501374e-01
 2.49644741e+01 1.00996595e+02 1.15010062e+01 9.64416630e+01
 4.71799744e+04 1.12813400e+03]
[8.61506927e+00 5.34268573e+02 7.79979905e+00 7.74187610e+00
 2.01804693e+01 9.33145221e+01 2.62178414e+03 8.34404091e+06
 4.12179292e+04 3.11120993e+03]


In [10]:
N=128
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.50636170e-01 2.51828551e-01 3.41618330e-01 8.13286927e-01
 1.84991731e-01 1.35610701e+01 1.30043054e+02 1.02664226e+03
 3.00127399e+00 1.18379684e+01]
[4.34859016e+00 5.12984808e+00 3.97684840e+00 3.75351808e+00
 3.78433738e+00 1.29759883e+03 2.61710395e+03 2.93950363e+06
 3.11723659e+06 3.26061075e+04]


In [ ]:
N = 32
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

In [ ]:
N = 16
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

In [ ]:
N = 8
cdl_est_values = train_cdl(N, 10)
dr_est_values = train_dr(N, 10)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

In [ ]:
N = 8
cdl_est_values = train_cdl(N, 100)
dr_est_values = train_dr(N, 100)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

# Training Step

In [11]:
def train_esm(N = 64, training_steps = 10, noise = 0.1):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        #dataset = create_dataset_DGP_binary_A_more_noise(Dim=Dim, N=N, noise = noise)
        dataset = create_dataset_DGP_binary_A_conf(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()
        
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()


        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values, cdl_est_values

In [12]:
for N in [8, 16, 32, 64, 128]:
    dr_est_values, cdl_est_values = train_esm(N, 10)
    print(np.array(cdl_est_values).mean(axis=0))
    print(np.array(dr_est_values).mean(axis=0))
    print("--"*5 + str(N)+"--"*5 )

[1.31547842e+01 1.73280756e-01 1.17183995e-01 1.82330843e+00
 2.07884861e-01 2.23116390e-01 7.78402554e+02 7.33982662e-01
 8.02943257e-01 1.03628662e+01]
[1.89929298e+01 2.66419522e-01 1.94221662e-01 2.45989381e+00
 2.87373597e-01 6.21806954e-01 2.70172439e+03 1.87946417e+00
 3.61373760e+00 4.22714500e+01]
----------8----------
[0.06898578 0.10982551 0.2617607  0.40477213 0.0435957  2.14088626
 0.73436396 0.77330426 0.37491311 0.23698735]
[0.08910666 0.07416968 0.40184182 1.48501316 0.11630445 0.46499816
 1.30036904 0.69127049 0.4901029  0.18706291]
----------16----------
[1.86766602e-01 9.64338481e-02 1.97127333e-01 1.23148128e-01
 1.89800043e+00 1.94825692e+01 3.57280501e+01 3.03296854e-01
 1.58314360e+03 2.94712819e+02]
[2.39874109e-02 3.08062576e-02 2.64425001e-01 3.90342452e-02
 6.42456384e-01 8.52548457e+00 4.79466914e+00 1.19641209e-01
 3.05708110e+02 4.41885638e+01]
----------32----------
[3.07536560e+00 2.76674908e-01 1.80680092e-01 1.83713320e-01
 1.76113516e-01 2.68694019e+0

In [13]:
for N in [8, 16, 32, 64, 128]:
    dr_est_values, cdl_est_values = train_esm(N, 20)
    print(np.array(cdl_est_values).mean(axis=0))
    print(np.array(dr_est_values).mean(axis=0))
    print("--"*5 + str(N)+"--"*5 )

[8.86353708e+02 2.79077705e+01 8.41839690e+01 8.36915318e-01
 1.78719174e+01 1.01420350e+03 5.12897903e+00 9.98290751e+01
 9.79660652e+01 6.48419108e+00]
[4.84332749e+02 3.12275785e+01 5.32818646e+01 9.99005201e-01
 1.63977099e+01 7.80220060e+03 2.64307278e+00 2.63723593e+02
 2.71248675e+02 1.39402335e+01]
----------8----------
[2.56413789e+02 2.31900303e+01 3.37180200e+04 1.00431359e+01
 3.14599085e+00 1.72148926e+05 6.55660335e+02 2.67861362e+04
 4.50546014e+05 1.36969441e+02]
[2.79942222e+02 3.00418230e+01 8.79066996e+03 7.90588871e+00
 4.62475050e+00 1.05478029e+05 2.69823205e+02 1.88954219e+04
 1.92884941e+05 5.30807209e+01]
----------16----------
[2.23103183e-01 6.08677365e-01 1.80229026e-01 2.51212809e-01
 3.51774334e-01 3.57394440e-01 4.31625258e+01 5.13008160e+03
 1.00013555e+01 7.02303244e+00]
[5.49658958e-01 2.51684258e+00 2.32722645e-01 7.11358703e-01
 9.51727718e-01 3.01233111e+00 8.13888787e+01 1.01020858e+04
 1.42438103e+01 1.27897009e+01]
----------32----------
[8.39544